## Task 1 -- Web Search Agents: Implement any 3 web search agents with error handling and fallbacks


In [5]:
%pip install -qU langchain langchain-huggingface langchain-cohere sentence-transformers langgraph "langchain[google-genai]" langchain_tavily langchain_community

Note: you may need to restart the kernel to use updated packages.


In [6]:
from __future__ import annotations
import os, time, requests
from typing import List, Dict, Optional

from bs4 import BeautifulSoup
from pydantic import BaseModel

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

from langchain_cohere import ChatCohere

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
SERPAPI_KEY = os.environ["SERPAPI_KEY"]

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY, 
    temperature=0
)


In [8]:
# 1. Tavily Agent
from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    api_key=TAVILY_API_KEY
)

tavily_agent = create_react_agent(llm, [tavily_search_tool])

# 2. SerpAPI Agent
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool

serp = SerpAPIWrapper(
    serpapi_api_key=SERPAPI_KEY
)
serp_tool = Tool(
    name="serpapi-search",
    func=serp.run,
    description="Search engine powered by SerpAPI"
)

serp_agent = create_react_agent(llm, [serp_tool])

# 3. DuckDuckGO Agent
from langchain_community.tools import DuckDuckGoSearchRun

duckduckgo_tool = DuckDuckGoSearchRun()

ddg_agent = create_react_agent(llm, [duckduckgo_tool])


# Combined Agent
agent = create_react_agent(llm, [tavily_search_tool, duckduckgo_tool, serp_tool])


In [9]:
def safe_query(user_input: str):
    """Query the multi-agent with fallback + markdown pretty print."""
    try:
        print(f"\n## Query: {user_input}\n")
        for step in agent.stream({"messages": user_input}, stream_mode="values"):
            step["messages"][-1].pretty_print()
    except Exception as e:
        print(f"\n **Primary agent failed:** {e}\n")
        # Fallback: DuckDuckGo
        try:
            print("### Fallback: DuckDuckGo Search\n")
            print(duckduckgo_tool.run(user_input))
        except Exception as e2:
            print(f" **All search methods failed:** {e2}")


# Example run
safe_query("What nation hosted Euro 2024? Include only Wikipedia sources.")



## Query: What nation hosted Euro 2024? Include only Wikipedia sources.

================================ Human Message =================================

What nation hosted Euro 2024? Include only Wikipedia sources.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (dced30b9-5446-49ba-b371-a6407a52214e)
 Call ID: dced30b9-5446-49ba-b371-a6407a52214e
  Args:
    include_domains: ['en.wikipedia.org']
    query: Euro 2024 host nation
================================= Tool Message =================================
Name: tavily_search

{"query": "Euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "title": "UEFA Euro 2024 - Wikipedia", "content": "It was the third time that European Championship matches were played on German territory, and the second time in reunified Germany, as West Germany hosted the 1988 tournament, and four matc